In [10]:
# import pandas, numpy, matplotlib, seaborn 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# importing the requests library
import requests

### Resources
 1. [Restaurants in Wake County Data Info](https://www.arcgis.com/home/item.html?id=124c2187da8c41c59bde04fa67eb2872)
 2. [Wake County Open Data](https://data-wake.opendata.arcgis.com/search?tags=restaurants)
 3. [Food Inspection Violations Data Info](https://data.wakegov.com/datasets/Wake::food-inspection-violations/about)
 4. [Wake County Yelp Initiative](https://ash.harvard.edu/news/wake-county-yelp-initiative)

In [11]:
# https://data-wake.opendata.arcgis.com/datasets/food-inspections/explore
# https://www.arcgis.com/home/item.html?id=ebe3ae7f76954fad81411612d7c4fb17
def getFoodInspectionsDf(): 
    # https://maps.wakegov.com/arcgis/rest/services/Inspections/RestaurantInspectionsOpenData/MapServer/1/query?outFields=*&where=1%3D1
    
    # Sending get request and saving the response as response object
    # extracting data in json
    val = 'https://opendata.arcgis.com/datasets/ebe3ae7f76954fad81411612d7c4fb17_1.geojson'
    rows = []
    r = requests.get(url = val)
    data = r.json()
    #print(data)
    for d in data:
        data = r.json()['features']
        for d in data:
            rows.append(d['properties'])
    return pd.DataFrame(rows)

food_inspections_raw = getFoodInspectionsDf()

In [20]:
food_inspections_raw.head()

,OBJECTID,HSISID,SCORE,DATE_,DESCRIPTION,TYPE,INSPECTOR,PERMITID
0,21522530,04092017542,94.5,2017-04-07T00:00:00Z,"Inspection conducted by Joshua Volkan, supervi...",Inspection,Anne-Kathrin Bartoli,1
1,21522531,04092017542,92.0,2017-11-08T00:00:00Z,manager owns two deep chest freezers that are ...,Inspection,Laura McNeill,1
2,21522532,04092017542,95.0,2018-03-23T00:00:00Z,None,Inspection,Laura McNeill,1
3,21522533,04092017542,93.5,2018-09-07T00:00:00Z,"*NOTICE* EFFECTIVE JANUARY 1, 2019, THE NC FOO...",Inspection,Laura McNeill,1
4,21522534,04092017542,93.0,2019-04-04T00:00:00Z,"*NOTICE* AS OF JANUARY 1, 2019, THE NC FOOD CO...",Inspection,Joanne Rutkofske,1


In [56]:
def preprocess_inspections(df):
        
    # drop duplicates
    df.drop_duplicates(inplace=True)
                       
    # convert date to datetime & only get date
    df['DATE_'] = pd.to_datetime(df['DATE_']).dt.date
    df.rename(columns={"DATE_": "DATE"}, inplace=True)

    
    # filter only since beginning of 2019 - we don't want data before that
    df = df[df['DATE'] >= pd.to_datetime('2019-01-01').date()].reset_index(drop=True)
    
    return df

In [57]:
df = preprocess_inspections(food_inspections_raw.copy())

In [58]:
print(df.shape)
df.head(30)

(20681, 8)


,OBJECTID,HSISID,SCORE,DATE,DESCRIPTION,TYPE,INSPECTOR,PERMITID
0,21522534,04092017542,93.0,2019-04-04,"*NOTICE* AS OF JANUARY 1, 2019, THE NC FOOD CO...",Inspection,Joanne Rutkofske,1
1,21522535,04092017542,93.5,2019-10-07,Follow-Up: 10/17/2019,Inspection,Naterra McQueen,1
2,21522536,04092017542,92.5,2020-05-19,"*NOTICE* AS OF JANUARY 1, 2019, THE NC FOOD CO...",Inspection,Naterra McQueen,1
3,21522537,04092017542,94.0,2020-10-09,PIC cannot sign due to COVID-19 concerns.,Inspection,Nicole Millard,1
4,21522538,04092017542,94.0,2021-03-24,PIC cannot sign due to COVID-19 concerns.,Inspection,Nicole Millard,1
5,21522539,04092017542,92.0,2021-07-20,Bathrooms for customers were locked today but ...,Inspection,David Adcock,1
6,21522551,04092015321,95.5,2019-04-24,"*NOTICE* AS OF JANUARY 1, 2019, THE NC FOOD CO...",Inspection,Sarah Thompson,377
7,21522552,04092015321,96.5,2019-11-05,"*NOTICE* AS OF JANUARY 1, 2019, THE NC FOOD CO...",Inspection,Joanne Rutkofske,377
8,21522553,04092015321,97.5,2020-04-09,"*NOTICE* AS OF JANUARY 1, 2019, THE NC FOOD CO...",Inspection,Joanne Rutkofske,377
9,21522554,04092015321,97.5,2020-08-04,"*NOTICE* AS OF JANUARY 1, 2019, THE NC FOOD CO...",Inspection,Joanne Rutkofske,377


In [59]:
display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20681 entries, 0 to 20680
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   OBJECTID     20681 non-null  int64  
 1   HSISID       20681 non-null  object 
 2   SCORE        20681 non-null  float64
 3   DATE         20681 non-null  object 
 4   DESCRIPTION  16621 non-null  object 
 5   TYPE         20681 non-null  object 
 6   INSPECTOR    20681 non-null  object 
 7   PERMITID     20681 non-null  int64  
dtypes: float64(1), int64(2), object(5)
memory usage: 1.3+ MB


None

In [60]:
df.isna().mean()

OBJECTID       0.000000
HSISID         0.000000
SCORE          0.000000
DATE           0.000000
DESCRIPTION    0.196315
TYPE           0.000000
INSPECTOR      0.000000
PERMITID       0.000000
dtype: float64

In [61]:
display({column: len(df[column].unique()) for column in df.columns})

{'OBJECTID': 20681,
 'HSISID': 3862,
 'SCORE': 48,
 'DATE': 770,
 'DESCRIPTION': 5928,
 'TYPE': 2,
 'INSPECTOR': 47,
 'PERMITID': 3862}

In [62]:
df.dtypes

OBJECTID         int64
HSISID          object
SCORE          float64
DATE            object
DESCRIPTION     object
TYPE            object
INSPECTOR       object
PERMITID         int64
dtype: object